In [8]:
# import
import pandas as pd
import numpy as np
import random
import os
from datetime import datetime
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
# import torch.nn.functional as F # 비선형석 첨가
from torch.utils.data import Dataset, DataLoader, TensorDataset

from tqdm.auto import tqdm

from matplotlib import font_manager,rc
rc('font', family='AppleGothic')
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings(action='ignore')

In [9]:
# 데이터 불러오기
train_df_pass = './processed_data/train_df_dl.pkl'
test_df_pass = './processed_data/test_df_dl.pkl'
# train_df_pass = './processed_data/train_df_dl_win.pkl'
# test_df_pass = './processed_data/test_df_dl_win.pkl'
train_df = pd.read_pickle(train_df_pass)
test_df = pd.read_pickle(test_df_pass)

display(train_df)
display(test_df)

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,year,month,day,hour,day_of_week,day_of_year,WCT,THI,holiday,building_type,total_area,cooling_area,power_consumption
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,2022,6,1,0,2,152,20.584184,49.6576,1,Other_Buildings,110634.00,39570.00,1085.28
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,2022,6,1,1,2,152,19.846954,47.7625,1,Other_Buildings,110634.00,39570.00,1047.36
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,2022,6,1,2,2,152,19.317610,47.2225,1,Other_Buildings,110634.00,39570.00,974.88
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,2022,6,1,3,2,152,18.337658,44.7856,1,Other_Buildings,110634.00,39570.00,953.76
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,2022,6,1,4,2,152,19.585934,49.0061,1,Other_Buildings,110634.00,39570.00,986.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,2022,8,24,19,2,236,25.094857,63.6624,0,Hotel_and_Resort,57497.84,40035.23,881.04
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,2022,8,24,20,2,236,24.244918,62.4024,0,Hotel_and_Resort,57497.84,40035.23,798.96
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,2022,8,24,21,2,236,23.241700,64.1976,0,Hotel_and_Resort,57497.84,40035.23,825.12
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,2022,8,24,22,2,236,23.472275,65.0744,0,Hotel_and_Resort,57497.84,40035.23,640.08


,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,year,month,day,hour,day_of_week,day_of_year,WCT,THI,holiday,building_type,total_area,cooling_area
0,1_20220825 00,1,2022-08-25 00:00:00,23.5,0.0,2.2,72,2022,8,25,0,3,237,25.185563,58.3456,0,Other_Buildings,110634.00,39570.00
1,1_20220825 01,1,2022-08-25 01:00:00,23.0,0.0,0.9,72,2022,8,25,1,3,237,24.994620,57.4456,0,Other_Buildings,110634.00,39570.00
2,1_20220825 02,1,2022-08-25 02:00:00,22.7,0.0,1.5,75,2022,8,25,2,3,237,24.495488,57.8725,0,Other_Buildings,110634.00,39570.00
3,1_20220825 03,1,2022-08-25 03:00:00,22.1,0.0,1.3,78,2022,8,25,3,3,237,23.937118,57.9376,0,Other_Buildings,110634.00,39570.00
4,1_20220825 04,1,2022-08-25 04:00:00,21.8,0.0,1.0,77,2022,8,25,4,3,237,23.746200,56.9961,0,Other_Buildings,110634.00,39570.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100_20220831 19,100,2022-08-31 19:00:00,22.5,0.0,0.9,84,2022,8,31,19,2,243,24.493434,61.4824,0,Hotel_and_Resort,57497.84,40035.23
16796,100_20220831 20,100,2022-08-31 20:00:00,20.7,0.0,0.4,95,2022,8,31,20,2,243,23.067551,65.2725,0,Hotel_and_Resort,57497.84,40035.23
16797,100_20220831 21,100,2022-08-31 21:00:00,20.2,0.0,0.4,98,2022,8,31,21,2,243,22.590086,66.7056,0,Hotel_and_Resort,57497.84,40035.23
16798,100_20220831 22,100,2022-08-31 22:00:00,20.1,0.0,1.1,97,2022,8,31,22,2,243,21.978649,65.7281,0,Hotel_and_Resort,57497.84,40035.23


In [10]:
# building_type 컬럼과 day_of_week 컬럼 원-핫 인코딩

# building_type용 OneHotEncoder 객체 생성
encoder = OneHotEncoder()

# building_type 컬럼을 원-핫 인코딩
encoded_data_train = encoder.fit_transform(train_df[['building_type']])
encoded_data_test = encoder.transform(test_df[['building_type']])

# 인코딩 결과를 데이터프레임으로 변환
df_encoded_train = pd.DataFrame(encoded_data_train.toarray(), columns=encoder.get_feature_names_out(['building_type']))
df_encoded_test = pd.DataFrame(encoded_data_test.toarray(), columns=encoder.get_feature_names_out(['building_type']))

train_df = pd.concat([train_df, df_encoded_train],axis=1)
test_df = pd.concat([test_df, df_encoded_test],axis=1)
# 기존 컬럼 drop
train_df = train_df.drop(columns=['building_type'])
test_df = test_df.drop(columns=['building_type'])

In [11]:
# day_of_week용 OneHotEncoder 객체 생성
encoder = OneHotEncoder()

# day_of_week 컬럼을 원-핫 인코딩
encoded_data_train = encoder.fit_transform(train_df[['day_of_week']])
encoded_data_test = encoder.transform(test_df[['day_of_week']])

# 인코딩 결과를 데이터프레임으로 변환
df_encoded_train = pd.DataFrame(encoded_data_train.toarray(), columns=encoder.get_feature_names_out(['day_of_week']))
df_encoded_test = pd.DataFrame(encoded_data_test.toarray(), columns=encoder.get_feature_names_out(['day_of_week']))

# 인코딩 결과를 데이터에 추가
train_df = pd.concat([train_df, df_encoded_train],axis=1)
test_df = pd.concat([test_df, df_encoded_test],axis=1)

# 기존 컬럼 drop
train_df = train_df.drop(columns=['day_of_week'])
test_df = test_df.drop(columns=['day_of_week'])

In [12]:
temp = train_df.pop('power_consumption')
train_df['power_consumption'] = temp

print(len(train_df.columns))
train_df.columns, test_df.columns

37


(Index(['num_date_time', 'building_number', 'date_time', 'temperature',
        'rainfall', 'windspeed', 'humidity', 'year', 'month', 'day', 'hour',
        'day_of_year', 'WCT', 'THI', 'holiday', 'total_area', 'cooling_area',
        'building_type_Apartment', 'building_type_Commercial',
        'building_type_Data_Center',
        'building_type_Department_Store_and_Outlet',
        'building_type_Discount_Mart', 'building_type_Hospital',
        'building_type_Hotel_and_Resort',
        'building_type_Knowledge_Industry_Center',
        'building_type_Other_Buildings', 'building_type_Public',
        'building_type_Research_Institute', 'building_type_University',
        'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3',
        'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'power_consumption'],
       dtype='object'),
 Index(['num_date_time', 'building_number', 'date_time', 'temperature',
        'rainfall', 'windspeed', 'humidity', 'year', 'month', 'day', 'ho

In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [36]:
# 하이퍼파라미터
param_grid = {
    'hidden_size' : [32, 64, 128],
    'num_layers' : 2,
    'output_size' : 1,
    'num_epochs' : 5,
    }
learning_rate = 0.001
input_size = 29
window_size = 24
batch_size = 64

In [39]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size):
        self.df = df
        self.window_size = window_size

    def __len__(self): # row의 길이 반환
        return len(self.df) - self.window_size

    def __getitem__(self, idx): # dataset의 indexing
        x = self.df[idx:idx+self.window_size, :].astype('float64')
        if self.df.shape[1] > 1:
            y = self.df[idx+self.window_size, -1].astype('float64')
        else:
            y = None
        return x, y

def create_data_loader(df, window_size, batch_size):
    dataset = TimeSeriesDataset(df, window_size)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [40]:
# normalization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df.drop(['num_date_time', 'building_number', 'date_time', 'year', 'month', 'day', 'hour', 'day_of_year'], axis=1).values)

train_loader = create_data_loader(train_data, window_size, batch_size)

In [41]:
# 데이터 분할: 훈련 세트와 테스트 세트
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성
model = xgb.XGBClassifier()

# GridSearchCV를 위한 하이퍼파라미터 그리드 정의
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [3, 4, 5],
#     'learning_rate': [0.01, 0.1, 0.2]
# }

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')

In [42]:
# Train
for epoch in range(param_grid['num_epochs']):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs
        labels = labels.unsqueeze(1)

        grid_search.fit(inputs, labels)

        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
            
print('==================== 학습 완료 ====================')

TypeError: Parameter grid for parameter 'num_layers' needs to be a list or a numpy array, but got 2 (of type int) instead. Single values need to be wrapped in a list with one element.

In [ ]:
# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)

# 최적 모델로 테스트 데이터에 대한 예측 수행
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print("테스트 정확도:", accuracy)